# Projeto: Assistente de Viagem

Neste notebook vamos combinar todos os conceitos vistos ate agora para construir um agente funcional: um **assistente de viagem** que ajuda o usuario a planejar destinos.

O agente vai usar:

- **System prompt** para definir sua persona e comportamento
- **Tool** para buscar informacoes atualizadas na web
- **Memoria** para manter o contexto entre as mensagens

O objetivo e mostrar como esses blocos fundamentais se combinam para criar um agente util e funcional.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

: 

## Definindo a tool de busca

O assistente precisa buscar informacoes atualizadas sobre destinos, precos de passagens, clima e atracoes. Para isso, vamos reutilizar a tool de busca na web com Tavily.

In [ ]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def buscar_na_web(query: str) -> Dict[str, Any]:
    """Busca informacoes atualizadas na internet."""
    return tavily_client.search(query)

## Definindo o system prompt

Um bom system prompt define claramente o papel do agente, suas capacidades e o formato esperado das respostas. Quanto mais especifico o prompt, mais consistente e util sera o comportamento do agente.

In [ ]:
system_prompt = """
Voce e um assistente de viagem experiente e atencioso.

Seu papel e ajudar o usuario a planejar viagens, oferecendo sugestoes de destinos, roteiros, dicas praticas e informacoes uteis.

Regras:
- Sempre use a ferramenta de busca para obter informacoes atualizadas sobre precos, clima e eventos
- Seja objetivo e organize as informacoes de forma clara
- Quando sugerir roteiros, inclua estimativas de tempo e custo quando possivel
- Lembre-se das preferencias que o usuario mencionou durante a conversa
"""

## Criando o agente

Agora combinamos tudo: modelo, tools, system prompt e checkpointer para memoria.

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agente = create_agent(
    model="gpt-4.1-nano",
    tools=[buscar_na_web],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [ ]:
config = {"configurable": {"thread_id": "viagem-1"}}

## Interagindo com o agente

Vamos simular uma conversa real com o assistente. Cada chamada a seguir faz parte da mesma conversa (mesmo `thread_id`), entao o agente vai acumulando contexto.

In [ ]:
from langchain.messages import HumanMessage

resposta = agente.invoke(
    {"messages": [HumanMessage(content="Estou pensando em viajar para o Japao em outubro. O que voce sugere?")]},
    config
)

print(resposta["messages"][-1].content)

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Meu orcamento e de 15 mil reais, incluindo passagem. Da pra fazer?")]},
    config
)

print(resposta["messages"][-1].content)

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Monta um roteiro de 7 dias pra mim focando em Tokyo e Kyoto.")]},
    config
)

print(resposta["messages"][-1].content)

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Qual o meu orcamento mesmo? E pra quando eu quero ir?")]},
    config
)

print(resposta["messages"][-1].content)

Essa ultima pergunta confirma que a memoria esta funcionando. O agente lembra do orcamento e do periodo mencionados nas mensagens anteriores, sem que tenhamos passado essas informacoes novamente.

Com apenas algumas linhas de codigo, combinamos modelo, tools, prompting e memoria para criar um assistente funcional. Nas proximas aulas, vamos expandir esses conceitos com fluxos mais complexos usando LangGraph.